In [175]:
import os
import json
import pandas as pd
import datetime
import requests
import numpy as np
import espn_api

In [176]:
def readSecrets ():
    filename = os.path.join('secrets.json')
    try:
        with open(filename, mode='r') as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}

# Dictionary of {"swid": "", "espn_s2": ""}
secrets = readSecrets()
LEAGUE_ID = "699385"
allYears = list(range(2015,2023))
weeks = list(range(1,19))

In [177]:
scores = []
for year in allYears:
    league = League(league_id=LEAGUE_ID, year=year, espn_s2=secrets['espn_s2'], swid=secrets['swid'])
    for week in weeks:
        matchups = league.scoreboard(week)
        for matchup in matchups:
            if type(matchup.home_team) == espn_api.football.team.Team:
                scores.append([year, week, matchup.home_score, matchup.home_team.team_name])
            if type(matchup.away_team) == espn_api.football.team.Team:
                scores.append([year, week, matchup.away_score, matchup.away_team.team_name])


In [178]:
scores

[[2015, 1, 103.0, "Budda's Dozen"],
 [2015, 1, 112.0, "Marc's  Bird Hammer "],
 [2015, 1, 120.0, 'Ertz So Good'],
 [2015, 1, 104.0, 'Team berk'],
 [2015, 1, 120.0, 'North Dakota Redskin'],
 [2015, 1, 60.0, 'Sky Nakynadodichodi'],
 [2015, 1, 92.0, 'Team berk sucks'],
 [2015, 1, 102.0, 'Team Waller'],
 [2015, 1, 135.0, 'Washington Fighting Siouxs'],
 [2015, 1, 110.0, 'Damn it Kelce'],
 [2015, 2, 102.0, 'Team berk'],
 [2015, 2, 103.0, 'Sky Nakynadodichodi'],
 [2015, 2, 103.0, 'Team Waller'],
 [2015, 2, 79.0, "Budda's Dozen"],
 [2015, 2, 119.0, 'Damn it Kelce'],
 [2015, 2, 94.0, "Marc's  Bird Hammer "],
 [2015, 2, 107.0, 'Team berk sucks'],
 [2015, 2, 58.0, 'Ertz So Good'],
 [2015, 2, 97.0, 'Washington Fighting Siouxs'],
 [2015, 2, 107.0, 'North Dakota Redskin'],
 [2015, 3, 148.0, 'Team Waller'],
 [2015, 3, 101.0, 'Damn it Kelce'],
 [2015, 3, 75.0, 'Team berk'],
 [2015, 3, 155.0, 'Team berk sucks'],
 [2015, 3, 97.0, 'Sky Nakynadodichodi'],
 [2015, 3, 131.0, 'Washington Fighting Siouxs'],
 

In [179]:
df = pd.DataFrame.from_records(scores, columns = ['year', 'week', 'points', 'team'])
df.head()

,year,week,points,team
0,2015,1,103.0,Budda's Dozen
1,2015,1,112.0,Marc's Bird Hammer
2,2015,1,120.0,Ertz So Good
3,2015,1,104.0,Team berk
4,2015,1,120.0,North Dakota Redskin


In [180]:
df = df[df['points'] != 0.0]

In [181]:
fromMean = []
for year in df['year'].unique():
    selector = df.loc[df['year'] == year]['points']
    mean = np.mean(selector)
    dev = np.std(selector)
    for val in selector:
        fromMean.append((val - mean) / dev)
df['fromDev'] = fromMean
    

In [182]:
df.head()

,year,week,points,team,fromDev
0,2015,1,103.0,Budda's Dozen,-0.035808
1,2015,1,112.0,Marc's Bird Hammer,0.363908
2,2015,1,120.0,Ertz So Good,0.719212
3,2015,1,104.0,Team berk,0.008605
4,2015,1,120.0,North Dakota Redskin,0.719212


In [183]:
df.sort_values(by=['fromDev']).head(20)

,year,week,points,team,fromDev
290,2016,14,65.060,From Wentz It Came,-2.548333
159,2015,16,47.000,Sky Nakynadodichodi,-2.522932
222,2016,7,66.400,Wubba Lubba Dub Dub,-2.491545
139,2015,14,50.000,Sky Nakynadodichodi,-2.389693
957,2020,16,88.380,Impossumble Comeback,-2.385415
308,2016,15,71.070,'14 Raiders,-2.293636
626,2018,15,78.790,North Dakota Redskin,-2.232199
304,2016,15,72.960,Marry Luck Kill,-2.213540
370,2017,6,72.490,North Dakota Redskin,-2.196194
1188,2022,6,105.385,Brooklyn Backstabbers,-2.166694


In [184]:
df.sort_values(by=['fromDev'], ascending=False).head(20)

,year,week,points,team,fromDev
400,2017,9,203.38,Wubba Lubba Dub Dub,3.343345
724,2019,9,216.38,Marianne Williamson 2020,3.338157
572,2018,10,222.08,Trust The Process,3.266099
953,2020,16,246.07,The Harrington Football Team,3.098919
246,2016,9,198.00,From Wentz It Came,3.085505
576,2018,10,216.85,Cox McLeod,3.065414
29,2015,3,170.00,North Dakota Redskin,2.939858
77,2015,8,169.00,Team berk,2.895445
141,2015,15,167.00,Ertz So Good,2.806620
582,2018,11,208.48,Ghosts of Rex Grossman,2.744242
